In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import mpld3
import xarray as xr
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#import matplotlib.colors as mcolors
import seaborn as sns
from datetime import datetime
import math
import scipy
import scipy.cluster.hierarchy as sch
from math import fsum
from varname import nameof
from datetime import date
from scipy import stats
import os
from scipy.stats import pearsonr,spearmanr
import myfunctions as mf
import plotly.express as px

from importlib import reload

#Define some lists/arrays
depth =np.array([0.02, 0.04, 0.06, 0.08,0.12,0.16,0.20,0.24,0.28,0.32,0.36,0.40,0.44,0.54,0.64,0.74,0.84,0.94,1.04,1.14,2.39,4.676,7.635,11.140,15.115])
node_z = [0.01,0.04,0.09,0.16,0.26,0.40,0.587,0.80,1.06,1.36,1.70,2.08,2.50,2.99,3.58,4.27,5.06,5.95,6.94,8.03,9.795,13.328,19.483,28.871,41.998]
sec_pr_day=60*60*24
depth_30=np.array([0.02,0.04,0.06,0.08,0.10]) #layer depths to multiply with concentrations
depth_50=np.array([0.02,0.04,0.06,0.08,0.12,0.16,0.02]) #[meters]

pool_names=['LITm', 'LITs', 'SAPb', 'SAPf', 'EcM','AM', 'SOMp', 'SOMa', 'SOMc'] #'ErM', 'AM',
N_pool_names = ['N_LITm', 'N_LITs', 'N_SAPb', 'N_SAPf', 'N_EcM', 'N_AM','N_SOMp', 'N_SOMa', 'N_SOMc', 'NH4_sol','NH4_sorp','NO3']

PFT_names=["not_vegetated                           ",
    "needleleaf_evergreen_temperate_tree     ",
    "needleleaf_evergreen_boreal_tree        ",
    "needleleaf_deciduous_boreal_tree        ",
    "broadleaf_evergreen_tropical_tree       ",
    "broadleaf_evergreen_temperate_tree      ",
    "broadleaf_deciduous_tropical_tree       ",
    "broadleaf_deciduous_temperate_tree      ",
    "broadleaf_deciduous_boreal_tree         ",
    "broadleaf_evergreen_shrub               ",
    "broadleaf_deciduous_temperate_shrub     ",
    "broadleaf_deciduous_boreal_shrub        ",
    "c3_arctic_grass                         ",
    "c3_non-arctic_grass                     ",
    "c4_grass                                ",]

PFT_names=[i.strip() for i in PFT_names]

temp_colors =["blue","purple","red"]
prec_colors = ["goldenrod","lightseagreen","darkblue"]

In [ ]:
#COPY THIS TO NOTEBOOK, THE globals() THING DOES NOT WORK FROM HERE. 
def import_decomp(dir_list,file_ext,date_bool):
    site_data=[]
    decomp_names=[]
    dirs = dir_list
    path_of_the_directory = '/home/ecaas/soil_decomp/results/'

    for folder in dirs:
        ext = (folder+file_ext)
        for files in os.listdir(path_of_the_directory+folder):
            if files.endswith(ext):
                filename = os.fsdecode(files)  
                name = filename[:-len(ext)-1]
                if name[0:2] != 'NR':
                    name='NR'+name#+folder
                else:
                    name = name #+folder
                decomp_names.append(name)
                globals()[name+"_"+folder+file_ext[0:-3]] = xr.load_dataset(path_of_the_directory+folder+"/"+filename)

                globals()[name+"_"+folder+file_ext[0:-3]].attrs['site_name'] = name
                globals()[name+"_"+folder+file_ext[0:-3]].attrs['run_name'] = folder
                if date_bool:
                    mf.add_date_dim(globals()[name+"_"+folder+file_ext[0:-3]])

                site_data.append(globals()[name+"_"+folder+file_ext[0:-3]])
            else:
                continue
        site_data =  sorted(site_data,key=lambda x: x.site_name)
        decomp_names = sorted(decomp_names)
        #decomp_names: list of name strings
        #site_data: list of xarrays
    return decomp_names,site_data